In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.model_selection import GridSearchCV




In [41]:
#READING THE CSV FILE
dg=pd.read_csv('train.tsv',delimiter='	')
#SHUFFLING THE DATA
dg = dg.sample(frac=1).reset_index(drop=True)
df=dg.copy()
#testing=df[0:2535]

#testing.to_csv("testing1", sep='\t', encoding='utf-8')
#training=df[2535:-1]
#training.to_csv("training1", sep='\t', encoding='utf-8')


df.columns=["ID","label","statement","subject","speaker","speakers_job_title","state_info","job_affiliation","barely_true","false","half_true","mostly_true","pants_on_fire","context"]
df

,ID,label,statement,subject,speaker,speakers_job_title,state_info,job_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,768.json,barely-true,"John McCain said ""he wouldn't even sit down wi...",foreign-policy,joe-biden,U.S. senator,Delaware,democrat,11.0,10.0,21.0,16.0,4.0,St. Louis
1,3480.json,mostly-true,Oregon concealed gun applications include home...,"guns,privacy",kim-thatcher,state representative,Oregon,republican,0.0,0.0,0.0,1.0,0.0,a House floor session
2,5540.json,mostly-true,Says there have been well over 54 million abor...,"abortion,supreme-court,women",chris-smith,NaN,NaN,republican,0.0,0.0,0.0,1.0,0.0,a speech on the House floor
3,7312.json,half-true,"Today, about 40 percent of guns are purchased ...","criminal-justice,guns",kirsten-gillibrand,U.S. Senator,New York,democrat,0.0,0.0,1.0,3.0,0.0,an interview on MSNBC
4,10620.json,mostly-true,"Of the 10,000 Rhode Island children in charter...","children,diversity,education",erika-sanzi,Homemaker,Rhode Island,activist,0.0,0.0,0.0,1.0,0.0,a newspaper commentary
5,201.json,true,The mayor said 'if you happen to be in this co...,immigration,mitt-romney,Former governor,Massachusetts,republican,34.0,32.0,58.0,33.0,19.0,"a debate in St. Petersburg, Fla."
6,8682.json,barely-true,Says Marco Rubio endorsed Obamacare for his ow...,health-care,charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an op-ed
7,1191.json,barely-true,Health insurance companies pay CEOs $24 millio...,"corporations,health-care",health-care-america-now,health care reform advocacy group,"Washington, D.C.",none,2.0,1.0,0.0,0.0,0.0,a television advertisement
8,7078.json,mostly-true,Texas population is projected to double in the...,"population,water",kirk-watson,lawyer,Texas,democrat,0.0,0.0,0.0,2.0,0.0,"an answer for the League of Women Voters, Aust..."
9,11950.json,pants-fire,The people that perpetrated that travesty in S...,terrorism,bill-clinton,former president,Arkansas,democrat,4.0,2.0,12.0,11.0,3.0,a campaign event


In [3]:
unique=set(df["label"])
print(unique)

{'barely-true', 'pants-fire', 'mostly-true', 'half-true', 'true', 'false'}


In [4]:
category3=df[df.label=="half-true"]
print(category3.shape)


del df["ID"]

(2114, 14)


In [5]:
df.speakers_job_title.replace(np.NaN, 'unknown', inplace=True)
df.state_info.replace(np.NaN, 'unknown', inplace=True)

In [6]:
df.dropna(inplace=True)
df.isnull().sum()

label                 0
statement             0
subject               0
speaker               0
speakers_job_title    0
state_info            0
job_affiliation       0
barely_true           0
false                 0
half_true             0
mostly_true           0
pants_on_fire         0
context               0
dtype: int64

In [7]:
def replace (s):
    if s=="mostly-true":
        return 0
    elif s=="barely-true":
        return 1
    elif s=="half-true":
        return 2
    elif s=="true":
        return 3
    elif s=="false":
        return 4
    elif s=="pants-fire":
        return 5
   
    
df["output"]=df.label.apply(replace)
output=df["output"]
unique=set(df["output"])
print(unique)

{0, 1, 2, 3, 4, 5}


In [8]:
#unique2=set(df["speaker"])
#len(unique2)
#SPEAKER
speaker=df["speaker"].values
unique2=set(speaker)
unique2=list(unique2)
dictionary2={}
count=0
for i in range(len(unique2)):

    dictionary2[unique2[i]]=count
    count+=1
    
speaker_replace=[]
for j in range (len(speaker)):
    speaker_replace.append(dictionary2[speaker[j]])
speaker_replace=np.array(speaker_replace)
speaker_replace=speaker_replace.reshape(10137,1)



#SPEAKER JOB TITLE
speaker_job=df["speakers_job_title"].values
unique3=set(speaker_job)
unique3=list(unique3)
dictionary3={}
count=0
for i in range(len(unique3)):

    dictionary3[unique3[i]]=count
    count+=1
    
speaker_job_replace=[]
for j in range (len(speaker_job)):
    speaker_job_replace.append(dictionary3[speaker_job[j]])
speaker_job_replace=np.array(speaker_job_replace)
speaker_job_replace=speaker_job_replace.reshape(10137,1)



#STATE INFO

state_info=df["state_info"].values
unique4=set(state_info)
unique4=list(unique4)
dictionary4={}
count=0
for i in range(len(unique4)):

    dictionary4[unique4[i]]=count
    count+=1
    
state_info_replace=[]
for j in range (len(state_info)):
    state_info_replace.append(dictionary4[state_info[j]])
state_info_replace=np.array(state_info_replace)
state_info_replace=state_info_replace.reshape(10137,1)


#job_affiliation

job_affiliation=df["job_affiliation"].values
unique5=set(job_affiliation)
unique5=list(unique5)
dictionary5={}
count=0
for i in range(len(unique5)):

    dictionary5[unique5[i]]=count
    count+=1
    
job_affiliation_replace=[]
for j in range (len(job_affiliation)):
    job_affiliation_replace.append(dictionary5[job_affiliation[j]])
job_affiliation_replace=np.array(job_affiliation_replace)
job_affiliation_replace=job_affiliation_replace.reshape(10137,1)



In [9]:
#CONTEXT

context=df["context"].values
unique6=set(context)
unique6=list(unique6)
dictionary6={}
count=0
for i in range(len(unique6)):

    dictionary6[unique6[i]]=count
    count+=1
    
context_replace=[]
for j in range (len(context)):
    context_replace.append(dictionary6[context[j]])
context_replace=np.array(context_replace)
context_replace=context_replace.reshape(10137,1)
context_replace

array([[  47],
       [ 862],
       [ 954],
       ...,
       [ 332],
       [3026],
       [3547]])

In [10]:
d=df["statement"]
statement=d.values
statement.shape
subject=df["subject"].values




CREATING A TOKENIZED FORM OF DATA TO WORK UPON

In [11]:
word_data=[]

for i in range(len(statement)):
    word_data.append(statement[i])
print(len(word_data))

10137


In [12]:
word_data1=[]

for i in range(len(subject)):
    word_data1.append(subject[i])
print(len(word_data1))

10137


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(word_data)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform(word_data)
# summarize encoded vector
print(vector.shape)
x=vector.toarray()



#ADDING COSINE SIMILARITIES

# create the transform
vectorizer1 = TfidfVectorizer()
# tokenize and build vocab
vectorizer1.fit(word_data)
# summarize
#print(vectorizer.vocabulary_)
#print(vectorizer.idf_)
# encode document
vector2 = vectorizer1.fit_transform(word_data)


from sklearn.metrics.pairwise import cosine_similarity
cosine2=cosine_similarity(vector2[0:1], vector2)

cosine2.shape



{'wikileaks': 11918, 'also': 955, 'shows': 9987, 'how': 5504, 'john': 6103, 'podesta': 8318, 'rigged': 9398, 'the': 10942, 'polls': 8349, 'by': 1979, 'oversampling': 7864, 'democrats': 3270, 'voter': 11691, 'suppression': 10669, 'technique': 10851, 'says': 9636, 'rep': 9194, 'charlie': 2264, 'bass': 1471, 'nh': 7474, 'voted': 11689, 'to': 11063, 'raise': 8870, 'his': 5388, 'own': 7888, 'pay': 8059, 'eight': 3859, 'times': 11051, 'under': 11374, 'obama': 7584, 'economy': 3826, 'utility': 11535, 'bills': 1623, 'are': 1155, 'higher': 5361, 'gov': 4996, 'rick': 9382, 'scott': 9720, 'signed': 10017, 'into': 5914, 'law': 6411, 'bill': 1617, 'that': 10934, 'gives': 4931, 'floridas': 4579, 'public': 8739, 'officials': 7660, 'an': 1008, 'easy': 3814, 'way': 11795, 'avoid': 1341, 'disclosing': 3486, 'their': 10956, 'personal': 8158, 'finances': 4488, 'cathedral': 2146, 'of': 7638, 'christ': 2332, 'king': 6268, 'is': 5985, 'one': 7697, 'archdiocese': 1150, 'atlantas': 1263, 'largest': 6383, 'and'

(10137, 12136)


(1, 10137)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(word_data1)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.fit_transform(word_data1)
# summarize encoded vector
print(vector.shape)
x1=vector.toarray()


#ADDING COSINE SIMILARITIES

# create the transform
vectorizer2 = TfidfVectorizer()
# tokenize and build vocab
vectorizer2.fit(word_data1)
# summarize
#print(vectorizer.vocabulary_)
#print(vectorizer.idf_)
# encode document
vector1 = vectorizer2.fit_transform(word_data1)


from sklearn.metrics.pairwise import cosine_similarity
cosine=cosine_similarity(vector1[0:1], vector1)

cosine.shape





{'polls': 123, 'congress': 38, 'voting': 169, 'record': 131, 'economy': 57, 'elections': 60, 'transparency': 163, 'religion': 135, 'foreign': 74, 'policy': 122, 'candidates': 26, 'biography': 19, 'criminal': 46, 'justice': 97, 'education': 58, 'environment': 62, 'legal': 100, 'issues': 94, 'ethics': 63, 'state': 151, 'budget': 22, 'message': 109, 'machine': 103, '2012': 1, 'abortion': 4, 'job': 95, 'accomplishments': 5, 'market': 105, 'regulation': 134, 'public': 129, 'health': 81, 'debates': 49, 'israel': 93, 'taxes': 157, 'corrections': 42, 'and': 13, 'updates': 166, 'care': 28, 'workers': 177, 'jobs': 96, 'iraq': 91, 'cap': 27, 'trade': 162, 'climate': 36, 'change': 31, 'guns': 79, 'supreme': 155, 'court': 44, 'energy': 61, 'immigration': 88, 'federal': 67, 'government': 78, 'efficiency': 59, 'bipartisanship': 20, 'small': 145, 'business': 24, 'gas': 76, 'prices': 127, 'county': 43, 'diversity': 54, 'homeland': 83, 'security': 142, 'terrorism': 159, 'debt': 50, 'deficit': 51, 'weath

(1, 10137)

In [15]:
x_x=np.hstack((x,x1))

In [16]:
cosine=cosine.reshape(10137,1)
cosine2=cosine2.reshape(10137,1)

x_x=np.hstack((x_x,cosine))
x_x=np.hstack((x_x,cosine2))

In [16]:
y=df["output"].values

NAIVE BAYES

In [34]:
from sklearn import model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_x, y, test_size = 0.25, random_state = 0)
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

train_score = clf.score(x_train, y_train)
test_score = clf.score(x_test, y_test)

print(test_score)
print(x_test.shape)
y_train=y_train.reshape(7602,1)
y_test=y_test.reshape(2535,1)
train1=np.hstack((x_train,y_train))
print(train1)
#test1=np.hstack((x_test,y_test))
#np.savetxt("train1.csv", train1, delimiter=",")
#np.savetxt("test1.csv", test1, delimiter=",")


0.24023668639053256
(2535, 12314)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         2.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.21661886 0.         ... 0.         0.         2.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         4.        ]]


RANDOM FOREST

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=10)

y_pred = rf_classifier.fit(x_train,y_train).predict(x_test)

rf_classifier.score(x_test, y_test)



0.2106508875739645